In [158]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os, sys

In [159]:
df = pd.read_csv('Churn_Modelling.csv')

In [160]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [161]:
X = X.drop(['RowNumber','CustomerId','Surname'], axis=1)

In [162]:
geography = pd.get_dummies(X['Geography'], drop_first=True)
gender = pd.get_dummies(X['Gender'], drop_first=True)

In [163]:
X= pd.concat([X,geography,gender], axis=1)

In [164]:
X= X.drop(['Geography','Gender'], axis=1)

In [165]:
#Split the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [166]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [167]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid


In [168]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i == 0:
            model.add(Dense(units = nodes, kernel_initializer = 'he_uniform',activation=activation,input_dim = X_train.shape[1]))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(units = nodes, kernel_initializer = 'he_uniform',activation=activation))
            model.add(Dropout(0.3))

    model.add(Dense(units=1, kernel_initializer='glorot_uniform', activation = 'sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

# def create_model(layers, activation):
#     model = Sequential()
#     for i, nodes in enumerate(layers):
#         if i==0:
#             model.add(Dense(nodes,input_dim=X_train.shape[1]))
#             model.add(Activation(activation))
#             model.add(Dropout(0.3))
#         else:
#             model.add(Dense(nodes))
#             model.add(Activation(activation))
#             model.add(Dropout(0.3))
            
#     model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
#     model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
#     return model

In [169]:
model = KerasClassifier(build_fn=create_model, verbose=0, layers = [[20], [40,20], [45,30,15]])

In [170]:
# layers = [[20], [40,20], [45,30,15]]
activations = ['sigmoid', 'relu']
param_grid = {
    'layers' : layers,
    'model__activation' : activations,
    'batch_size' : [128,256],
    'epochs' : [30]
}
# param_grid = dict(layers=layers, model__activation=activations, batch_size = [128, 256], epochs=[30])


In [171]:
grid_model = GridSearchCV(estimator= model, param_grid= param_grid, cv=5)

In [172]:
res = grid_model.fit(X_train, y_train)

c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\ChurnModelANN\venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\ChurnModelANN\venv\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\ChurnModelANN\venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\ChurnModelANN\venv\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\E0551793\OneDrive - Eaton\BLC-Saloni\BITSPilani\ChurnModelANN\venv\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\E0551793\OneDrive - Ea

In [173]:
print(res.best_score_, res.best_params_)

0.8456250000000001 {'batch_size': 128, 'epochs': 30, 'layers': [40, 20], 'model__activation': 'relu'}


In [174]:
y_pred = grid_model.predict(X_test)

In [175]:
y_pred = (y_pred > 0.5)

In [176]:
from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1548   47]
 [ 238  167]]


In [177]:
score = accuracy_score(y_test , y_pred)
print(score)

0.8575
